In [1]:
import os
import pandas as pd
import numpy as np
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/Bus-Capstone')
import ttools #homemade module
import gtfs #homemade module
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')
%matplotlib inline

In [2]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('gtfs/')
stops = gtfs.load_stops('gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('gtfs/')
print 'Finished loading GTFS data.'

Finished loading GTFS data.


In [3]:
# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('newdata_parsed.csv')#,parse_dates=dt_columns)
bustime.drop_duplicates(['vehicleID','RecordedAtTime'],inplace=True)
bustime['Trip'] = bustime['Trip'].str.replace('MTA NYCT_','')
bustime.set_index(['Line','Trip','TripDate','vehicleID','RecordedAtTime'],
                  inplace=True,drop=True,verify_integrity=True)

In [5]:
# for demonstration, use a subset. Just get data for one line (M5) on one day.
tripDateLookup = "2016-06-13"
bustime = bustime.xs((tripDateLookup),level=(2),drop_level=False)
# note that the AVL dataframe must be sorted by timestammp, since iloc[]
# selection is used later in this script to find the earliest time
bustime.sort_index(inplace=True)
print 'Finished loading BusTime data and and slicing one day.'

Finished loading BusTime data and and slicing one day.


In [7]:
len(trips.index)

178272

In [14]:
bustime_trip_ids = bustime.index.get_level_values(1).unique()

In [18]:
len(bustime_trip_ids)

52346

In [52]:
len(bustime.groupby(level=1).size())

52346

In [17]:
trips.loc[bustime_trip_ids].shape

(52346, 6)

In [44]:
service_ids = trips.loc[bustime_trip_ids]['service_id'].dropna().unique()

In [34]:
trips_reframed = trips.reset_index().set_index('service_id')

In [49]:
len(trips_reframed.loc[service_ids]['trip_id'].unique())

43365

In [53]:
trips_reframed.head()

,trip_id,block_id,direction_id,route_id,shape_id,trip_headsign
service_id,,,,,,
GH_B6-Weekday,GH_B6-Weekday-004000_BX12_1,NaN,1,BX12,BX120805,INWOOD BWY-207 ST- PEL PKY via FRDHAM RD
GH_B6-Weekday,GH_B6-Weekday-009000_BX12_1,NaN,0,BX12,BX120809,EDSON AV via FORDHAM RD via PELHAM PKY
GH_B6-Weekday,GH_B6-Weekday-013000_BX12_1,NaN,1,BX12,BX120792,INWOOD BWY-207 ST- PEL PKY via FRDHAM RD
GH_B6-Weekday,GH_B6-Weekday-017000_BX12_1,NaN,0,BX12,BX120809,EDSON AV via FORDHAM RD via PELHAM PKY
GH_B6-Weekday,GH_B6-Weekday-021000_BX12_1,NaN,1,BX12,BX120792,INWOOD BWY-207 ST- PEL PKY via FRDHAM RD


In [57]:
trips.loc['GH_B6-Weekday-013000_BX12_1']

block_id                                              NaN
direction_id                                            1
route_id                                             BX12
service_id                                  GH_B6-Weekday
shape_id                                         BX120792
trip_headsign    INWOOD BWY-207 ST- PEL PKY via FRDHAM RD
Name: GH_B6-Weekday-013000_BX12_1, dtype: object